# Analyze results for 3D images
Jan 21, 2021

In [1]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_3d_image_analysis import *

# sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/1_basic_GAN/1_main_code/')
# import post_analysis_pandas as post

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [5]:
img_size=64
dict1={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/512sq/'}


### Read validation data and compute metrics

In [6]:
# Get location for validation data
raw_data_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/'
input_files_dict={'64':raw_data_dir+'3d_data/dataset1_smoothing_const_params_64cube_100k/val.npy'}

In [7]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_plot_spectrum_3d(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1

### Extract validation data
num_bkgnd=1000
fname=input_files_dict[str(img_size)]
s_val=np.load(fname,mmap_mode='r')[:num_bkgnd][:,0,:,:]
s_val=f_transform(s_val)
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)

# del(s_val)

(1000, 64, 64, 64)


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/modules_3d_image_analysis.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


### Read run data

In [8]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [9]:
u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)

In [10]:
# parent_dir=u.result
parent_dir=dict1[str(img_size)]

dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20210405_175036_3d_l2.0_80k', '20210419_122229_train…

In [11]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210419_131248_3dgan


## Plot Losses

In [12]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [13]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time
1591,1591.0,49.0,0.265201,0.294575,0.559776,7.416818,122.432854,115.016037,2.624844,111.482971,2.611461,NaN,NaN,7.625783,-9.095711,-7.415527,0.888047
1592,1592.0,49.0,0.247113,0.208723,0.455836,4.490647,109.035309,104.544662,2.656614,107.110870,2.608716,NaN,NaN,9.025576,-5.531197,-4.473441,0.901363
1593,1593.0,49.0,0.270456,0.200112,0.470568,4.583757,114.976151,110.392395,2.717838,110.725128,2.749611,NaN,NaN,9.058159,-3.135375,-4.563468,0.892942
1594,1594.0,49.0,0.252388,0.312395,0.564782,7.831779,116.955383,109.123604,2.706482,109.857864,2.578518,NaN,NaN,8.195270,-1.428504,-7.831304,0.883533
1595,1595.0,49.0,0.134758,0.206894,0.341652,8.590085,114.065445,105.475357,2.662630,107.818390,2.464584,NaN,NaN,4.331258,-6.369500,-8.589773,0.899191
1596,1596.0,49.0,0.263210,0.243114,0.506325,6.839170,113.730247,106.891075,2.677587,108.282028,2.689176,NaN,NaN,2.248441,-7.681521,-6.836674,0.888524
1597,1597.0,49.0,0.190655,0.189900,0.380555,4.716389,110.401566,105.685181,2.640210,107.292343,2.629696,NaN,NaN,3.337369,-6.134415,-4.698703,0.905796
1598,1598.0,49.0,0.139409,0.148357,0.287766,3.559382,109.978401,106.419022,2.670332,107.702621,2.683523,NaN,NaN,4.009667,-4.179992,-3.507233,0.889206
1599,1599.0,49.0,0.157746,0.213751,0.371497,5.510996,115.128716,109.617722,2.704972,110.487755,2.790879,NaN,NaN,5.187134,-2.285548,-5.503902,0.888030
1600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109.859596,2.638385,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [17]:

chi=df_metrics.quantile(q=0.005,axis=0)['hist_chi']
print(chi)
df_metrics[df_metrics['hist_chi']<=chi].sort_values(by=['hist_chi']).head(10)

-2.2679277002811435


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time
207,207.0,6.0,0.218218,0.162273,0.380491,4.594465,113.186363,108.591896,-2.833032,109.582588,-2.876089,NaN,NaN,4.240169,-4.571784,-4.580982,0.891880
208,208.0,6.0,0.187813,0.128440,0.316253,3.480589,114.595222,111.114632,-2.842308,111.234970,-2.824240,NaN,NaN,4.202448,-3.330960,-3.444937,0.898617
205,205.0,6.0,0.299693,0.258011,0.557704,4.925722,113.104507,108.178787,-2.617266,107.425720,-2.816558,NaN,NaN,4.537111,-1.538757,-4.914130,0.895705
511,511.0,15.0,0.320531,0.732890,1.053420,4.786280,116.195839,111.409561,-1.360994,111.775116,-2.486925,NaN,NaN,2.303424,-0.214512,-4.760625,0.892994
203,203.0,6.0,0.224735,0.143734,0.368468,4.647406,113.963867,109.316460,-2.158445,108.678383,-2.483051,NaN,NaN,5.347981,-2.977457,-4.634480,0.893663
520,520.0,16.0,0.167502,0.219865,0.387367,3.540251,114.346527,110.806274,-2.097867,107.534225,-2.373573,NaN,NaN,3.672894,-2.520154,-3.454246,0.892616
206,206.0,6.0,0.309670,0.170970,0.480640,5.584869,116.188644,110.603775,-2.643845,115.960693,-2.331090,NaN,NaN,3.434217,-4.537729,-5.580170,0.897029
519,519.0,16.0,0.360938,0.158434,0.519372,2.850835,111.677780,108.826942,-2.666097,110.302742,-2.315621,NaN,NaN,1.720856,-2.992725,-2.716249,0.891933


In [18]:
# display(df_metrics.sort_values(by=['hist_chi']).head(8))
# display(df_metrics.sort_values(by=['spec_chi']).head(8))

## Read stored chi-squares for images

In [19]:
df=pd.read_pickle(result_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)
df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting

col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']


In [20]:
def f_slice_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',head=10,epoch_range=[0,None],display_flag=False):
    ''' View dataframe after slicing
    '''
    col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']
    if (col_mode=='short'): col_list=['label','chi_1','chi_spec1']
        
    print(epoch_range)
    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ## Apply cutoff to keep reasonable chi1 and chispec1
    best_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[best_keys])   
    print(q_dict)
    df=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
        
    df2=df1.head(head)[col_list]

    if display_flag:   # Display df
        display_cols=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2']
        display(df2)
    
    return df2

In [21]:
w=interactive(f_slice_df,df=fixed(df),
cutoff=widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=['chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2'],
)
display(w)

interactive(children=(FloatSlider(value=0.2, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [22]:
df_sliced=w.result
# df_sliced

In [23]:
best_step=[]
df_test=df.copy()
# df_test=df[df.epoch>5]
best_step.append(f_slice_df(df_test,cutoff=0.9,sort_col='chi_1',head=4,display_flag=False).step.values)
best_step.append(f_slice_df(df_test,cutoff=0.8,sort_col='chi_1c',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df_test,cutoff=0.4,sort_col='chi_spec1',head=2,display_flag=False).step.values)
print(best_step)
best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[0, None]
{'chi_1': 1.5533178654225526, 'chi_spec1': 1927.5022688640038}
[0, 49]
{'chi_1': 0.205462890221009, 'chi_spec1': 962.8806836944607}
[0, 49]
{'chi_1': 0.146272865654264, 'chi_spec1': 121.25042451673932}
[array([ 260, 1440, 1470, 1230]), array([ 200, 1270]), array([1500, 1350])]
[ 200  260 1230 1270 1350 1440 1470 1500]


In [24]:
# best_step=[58800]
# best_step=np.arange(32800,32900,10).astype(int)

In [25]:
df_best=df[df.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(8, 22)
[(6, 200), (8, 260), (38, 1230), (39, 1270), (42, 1350), (44, 1440), (45, 1470), (46, 1500)]


In [26]:
df_best[col_list]


,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
19,6-200,0.076222,799.697230,52.590881,0.071650,0.004572,0.0,0.000822,6.088001,35014.828713,6,200
25,8-260,0.022973,1088.839546,52.137237,0.007385,0.015588,0.0,0.000014,8.360224,74869.688577,8,260
122,38-1230,0.045722,31.665181,49.472693,0.041150,0.004572,0.0,0.000207,3.369333,4689.943519,38,1230
126,39-1270,0.192943,897.523654,49.812498,0.188372,0.004572,0.0,0.001794,0.320710,12908.525676,39,1270
134,42-1350,0.081230,24.852898,56.907352,0.076658,0.004572,0.0,0.000668,2.909651,1795.582733,42,1350
143,44-1440,0.035618,27.437536,52.232308,0.031046,0.004572,0.0,0.000191,3.677551,2279.391551,44,1440
146,45-1470,0.040991,24.989060,56.491963,0.036419,0.004572,0.0,0.000729,4.760301,2419.345881,45,1470
149,46-1500,0.090328,23.734970,56.513812,0.085756,0.004572,0.0,0.000450,2.137875,1253.947388,46,1500


### Plots 2: stored data

In [27]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    img_size=64
    assert plot_type in ['hist','spec','grid','spec_relative','hist_relative'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative','hist_relative']:     fig=plt.figure(figsize=(6,6))


    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)
            plt.xlim(0,img_size/2)


        if plot_type=='spec_relative':
            
            y2=row.spec_val
            yerr2=row.spec_sdev
            x2=np.arange(len(y2))
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r=sqrt(\delta_c2 ^ 2/ c1^2 + \delta_c1^2 * c_2^2 / c_1^4)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row['num_imgs'])

            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)            
            
            
        if plot_type=='hist_relative':

            x2=row.hist_bin_centers
            y2=row.hist_val
            yerr2=row.hist_err
            x2=f_invtransform(x2)
            
#             plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['hist_val'],dict_bkg['hist_err']

            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r=sqrt(\delta_c2 ^ 2/ c1^2 + \delta_c1^2 * c_2^2 / c_1^4)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row['num_imgs'])

            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)            
            plt.xlim(0,img_size/2)
            plt.ylim(0,5)
            plt.legend()

        if plot_type=='grid':
            images=np.load(row.fname)[:,:,:,0]
            print(images.shape)
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.xlim(0,img_size/2)
        plt.title('Spectrum')
        
    if plot_type=='spec_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
        plt.xlim(0,img_size/2)
        plt.ylim(0.5,2)
        plt.legend()
        
    if plot_type=='hist_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
        
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    if plot_type in ['hist','spec']:     plt.yscale('log')

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_best,dict_val,'grid')


In [28]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)
    
interact_manual(f_widget_compare,df=fixed(df_best),
                labels_list=SelectMultiple(options=df_best.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','grid','spec_relative','hist_relative']))

interactive(children=(SelectMultiple(description='labels_list', options=('6-200', '8-260', '38-1230', '39-1270…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

## Plot images

In [94]:
print(s_val.shape)


(1000, 64, 64, 64)


In [95]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210123_050147_3d_l0.5_80k/images/gen_img_epoch-28_step-71620.npy'
a1=np.load(fname)

In [96]:
s_val.shape,a1.shape

((1000, 64, 64, 64), (32, 64, 64, 64))

In [97]:
f_plot_grid(a1[6:14,:,:,0],cols=4,fig_size=(4,2))

2 4


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [98]:
f_plot_grid(s_val[100:108,:,:,0],cols=4,fig_size=(4,2))

2 4


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [99]:
df_best[col_list]

,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
655,2-6560,0.246885,10.027608,49.063214,0.192645,0.053953,0.000288,0.115025,3.904391,17956.069610,2,6560
2819,11-28200,0.036677,6.014820,49.570766,0.019308,0.014379,0.002990,0.010212,9.884761,1447.230356,11,28200
2920,11-29210,0.248065,6.946046,59.081433,0.241302,0.006456,0.000306,0.089805,33.648975,50.338523,11,29210
3378,13-33790,0.032535,6.317830,50.120442,0.028484,0.002187,0.001864,0.013548,6.696044,1461.858703,13,33790
4015,16-40160,0.032709,5.377288,50.452657,0.027667,0.003498,0.001544,0.013563,10.932140,240.525028,16,40160
4024,16-40250,0.100561,3.085800,43.490086,0.071879,0.025700,0.002983,0.051874,7.510610,71.322752,16,40250
4107,16-41080,0.159360,3.133683,47.506667,0.103007,0.053081,0.003272,0.063929,15.402410,45.373778,16,41080
4489,17-44900,0.023849,4.554960,53.793261,0.017225,0.005775,0.000849,0.004996,17.831019,80.620578,17,44900


In [100]:
dims = [2,3,4,7]
np_x = np.random.uniform(size=dims).astype(np.float32)

In [31]:
90000/(32*32)

87.890625

In [30]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset1_smoothing_const_params_64cube_100k/norm_1_train_val.npy'
np.load(fname,mmap_mode='r').shape


(101251, 1, 64, 64, 64)

In [32]:
! du -hs /global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset1_smoothing_const_params_64cube_100k/norm_1_train_val.npy



99G	/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset1_smoothing_const_params_64cube_100k/norm_1_train_val.npy
